You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan: 
Title: Unveiling the Latest Trends and Key Players in Artificial Intelligence

Introduction: 
- Brief overview of Artificial Intelligence (AI) and its impact on various industries
- Mention of the rapid advancements and innovations in AI technology

Key Points: 
1. Latest Trends in Artificial Intelligence:
- Machine learning and deep learning algorithms
- Natural Language Processing (NLP) and sentiment analysis
- AI-powered chatbots and virtual assistants
- Autonomous vehicles and robotics
- A

I now can give a great answer

Final Answer:
# Unveiling the Latest Trends and Key Players in Artificial Intelligence

## Introduction
Artificial Intelligence (AI) has become a transformative force across various industries, revolutionizing the way businesses operate and individuals interact with technology. The rapid advancements in AI technology have paved the way for innovative solutions and futuristic possibilities. From machine learning algorithms to AI-powered chatbots, the landscape of AI continues to evolve at an unprecedented pace.

## Latest Trends in AI
One of the prominent trends in AI is the widespread adoption of machine learning and deep learning algorithms, enabling machines to learn from data and make predictions. Natural Language Processing (NLP) and sentiment analysis have also gained traction, allowing AI systems to understand and interpret human language. AI-powered chatbots and virtual assistants have become increasingly prevalent in customer service and support f

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# Unveiling the Latest Trends and Key Players in Artificial Intelligence

## Introduction
Artificial Intelligence (AI) has become a transformative force across various industries, revolutionizing the way businesses operate and individuals interact with technology. The rapid advancements in AI technology have paved the way for innovative solutions and futuristic possibilities. From machine learning algorithms to AI-powered chatbots, the landscape of AI continues to evolve at an unprecedented pace.

## Latest Trends in AI
One of the prominent trends in AI is the widespread adoption of machine learning and deep learning algorithms, enabling machines to learn from data and make predictions. Natural Language Processing (NLP) and sentiment analysis have also gained traction, allowing AI systems to understand and interpret human language. AI-powered chatbots and virtual assistants have become increasingly prevalent in customer service and support functions, enhancing user experiences. Moreover, the development of autonomous vehicles and robotics showcases the potential of AI in shaping the future of transportation and automation. Industries such as healthcare, finance, and retail are leveraging AI technologies to drive efficiency, improve decision-making, and enhance customer engagement.

## Key Players in the AI Industry
Several key players dominate the AI industry, leading the way in innovation and research. Google's DeepMind has made significant strides in machine learning and AI research, while IBM's Watson platform offers advanced cognitive computing capabilities. Amazon's Alexa has become a household name in smart home devices, and Microsoft's Cortana powers intelligent personal assistants. Tesla's self-driving cars exemplify the integration of AI in the automotive sector, and Facebook continues to invest in AI research to enhance user experiences and platform functionalities.

## Noteworthy News in AI
Recent developments in AI have sparked discussions on ethics, regulations, and the societal impact of AI technologies. Tech giants are partnering with AI startups to drive innovation and address complex challenges, while the implications of AI on the job market and workforce remain a topic of debate. AI applications are also being explored in combating climate change and addressing global challenges, showcasing the potential of AI to drive positive change in the world.

## Audience Engagement and Call to Action
For tech enthusiasts, industry professionals, business owners, and students alike, staying informed about the latest trends and key players in AI is crucial. By understanding the impact of AI on industries and society, individuals can leverage AI technologies to drive innovation and create value in their respective fields. As AI continues to shape the future of technology and business, it is essential to stay updated on AI advancements and engage in discussions on the implications of AI on our daily lives.

In conclusion, Artificial Intelligence represents a dynamic and evolving field with endless possibilities and challenges. By exploring the latest trends, key players, and noteworthy news in AI, we can gain valuable insights into the future of technology and the transformative power of AI in shaping our world. Embracing AI technologies responsibly and ethically will be essential in harnessing the full potential of AI for the benefit of society and the advancement of humanity. Let's continue to explore, innovate, and collaborate in the exciting realm of Artificial Intelligence.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [ ]:
topic = "YOUR TOPIC HERE"
result = crew.kickoff(inputs={"topic": topic})

In [ ]:
Markdown(result)

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).